In [29]:
using Random
using DataFrames
using CSV

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 400
const MXV = 40
const MNV = 2

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

fighters_to_df (generic function with 1 method)

In [28]:
library = Array{Fighter}(undef, 100000)
for ii in 1:length(library)
    library[ii] = rand_rename(random_fighter())
end

## Is the library large enough to find counters to large fighter sets?

In [33]:
bs = Array{Fighter}(undef, 10)
for ii in 1:length(bs)
    bs[ii] = rand_rename(random_tournament_winner(8))
end

In [34]:
fighters_to_df(bs)

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Irys the Agile,14,17,3,7
2,Funny Wolf,10,9,15,17
3,Agile Adiline,20,14,4,2
4,Naughty Lucelia,15,20,4,2
5,Mionna the Scorpion,15,16,2,8
6,Elegant Yaliyah,14,11,16,2
7,Wonderous Pteranodon,16,2,22,8
8,Noble Enara,3,13,25,22
9,Anilah the Fish,19,9,11,2


In [35]:
a = pick_best(library, bs)

Fighter("Coder Nicolo", 18, 16, 4, 2)

In [36]:
eval_battle_list(a, bs)

8

## Do higher tournament winners have an advantage over the max entropy distribution?

In [166]:
sum = 0
for ii in 1:10000
    a = rand_rename(random_tournament_winner(1))
    b = rand_rename(random_tournament_winner(0))
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.082

In [167]:
sum = 0
for ii in 1:10000
    a = random_tournament_winner(1)
    b = random_tournament_winner(0)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.0582

In [168]:
sum = 0
for ii in 1:10000
    a = random_tournament_winner(2)
    b = random_tournament_winner(0)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.0909

In [169]:
sum = 0
for ii in 1:10000
    a = random_tournament_winner(3)
    b = random_tournament_winner(0)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.1541

In [170]:
sum = 0
for ii in 1:10000
    a = random_tournament_winner(3)
    b = random_tournament_winner(2)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.0615

In [171]:
sum = 0
for ii in 1:1000
    a = random_tournament_winner(4)
    b = random_tournament_winner(2)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/1000

0.094

In [172]:
sum = 0
for ii in 1:1000
    a = random_tournament_winner(4)
    b = random_tournament_winner(3)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/1000

0.056

In [178]:
sum = 0
for ii in 1:1000
    a = random_tournament_winner(5)
    b = random_tournament_winner(3)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/1000

0.069

In [179]:
a = rand_rename(random_tournament_winner(5))
b = rand_rename(random_tournament_winner(3))
println(a)
println(b)
res = eval_battle(a, b)
println(res)


Fighter("Friendly Dasher", 2, 14, 25, 23)
Fighter("Rough Malaiyah", 6, 8, 5, 40)
-1


In [213]:
sum = 0
a = random_tournament_winner(6)
for ii in 1:10000
    b = random_tournament_winner(0)
    # println(a)
    # println(b)
    res = eval_battle(a, b)
    # println(res)
    sum = sum + res
end
sum/10000

0.3421

In [214]:
rand_rename(a)

Fighter("Zesty Pigeon", 15, 10, 13, 5)